# Multi-Turn Conversations

Learn to maintain conversation context and build chatbots!

## The Problem

Each API call is stateless - the model doesn't remember previous messages.

**Solution:** Send conversation history with each request.

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

## Example 1: Two-Turn Conversation

In [ ]:
# First turn
messages = [
    {"role": "user", "content": "My name is Alice."}
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

assistant_reply = response.choices[0].message.content
print("Assistant:", assistant_reply)

# Add assistant's response to history
messages.append({"role": "assistant", "content": assistant_reply})

# Second turn - model now knows previous context
messages.append({"role": "user", "content": "What's my name?"})

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

print("\nAssistant:", response.choices[0].message.content)  # Should say "Alice"!

## Example 2: Conversation Manager Class

Let's build a reusable conversation manager.

In [ ]:
class ConversationManager:
    def __init__(self, system_prompt: str = "You are a helpful assistant.", model: str = "gpt-3.5-turbo"):
        self.model = model
        self.messages = [{"role": "system", "content": system_prompt}]
        self.client = OpenAI()
    
    def send(self, user_message: str) -> str:
        """Send a message and get response."""
        # Add user message
        self.messages.append({"role": "user", "content": user_message})
        
        # Get response
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages
        )
        
        # Add assistant response to history
        assistant_message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message
    
    def get_history(self):
        """Return conversation history."""
        return self.messages
    
    def clear(self):
        """Clear conversation history (keeps system prompt)."""
        system_msg = self.messages[0]
        self.messages = [system_msg]

# Test it
conv = ConversationManager()

print("User: I love programming.")
print("Assistant:", conv.send("I love programming."))

print("\nUser: What do I love?")
print("Assistant:", conv.send("What do I love?"))

## Context Window Management

Problem: Conversations can exceed the context window!

**Solution strategies:**
1. Truncate old messages
2. Summarize conversation periodically
3. Use sliding window (keep recent N messages)

In [ ]:
import tiktoken

class SmartConversationManager:
    def __init__(self, system_prompt: str = "You are a helpful assistant.", 
                 model: str = "gpt-3.5-turbo",
                 max_tokens: int = 4000):  # Reserve tokens for context
        self.model = model
        self.max_tokens = max_tokens
        self.messages = [{"role": "system", "content": system_prompt}]
        self.client = OpenAI()
        self.encoding = tiktoken.encoding_for_model(model)
    
    def count_tokens(self) -> int:
        """Count tokens in current conversation."""
        total = 0
        for msg in self.messages:
            total += len(self.encoding.encode(msg["content"]))
        return total
    
    def trim_history(self):
        """Remove old messages if exceeding token limit."""
        while self.count_tokens() > self.max_tokens and len(self.messages) > 2:
            # Keep system prompt, remove oldest user/assistant pair
            if len(self.messages) > 1:
                self.messages.pop(1)  # Remove second message (first after system)
    
    def send(self, user_message: str) -> str:
        """Send message with automatic history management."""
        self.messages.append({"role": "user", "content": user_message})
        
        # Trim if needed
        self.trim_history()
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages
        )
        
        assistant_message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message

# Test with token tracking
smart_conv = SmartConversationManager(max_tokens=200)  # Very small for demo

for i in range(5):
    response = smart_conv.send(f"Tell me a fact about number {i}.")
    print(f"Turn {i+1}: {smart_conv.count_tokens()} tokens")
    
print(f"\nFinal message count: {len(smart_conv.messages)}")
print("(Old messages were automatically removed!)")

## Conversation State Patterns

### Pattern 1: Stateful Chatbot

In [ ]:
class PersonalityBot:
    """Chatbot with persistent personality."""
    def __init__(self, personality: str):
        self.conversation = ConversationManager(
            system_prompt=f"You are a chatbot with this personality: {personality}"
        )
        self.turn_count = 0
    
    def chat(self, message: str) -> str:
        self.turn_count += 1
        response = self.conversation.send(message)
        print(f"[Turn {self.turn_count}]")
        print(f"You: {message}")
        print(f"Bot: {response}\n")
        return response

# Create a pirate bot!
bot = PersonalityBot("You are a friendly pirate who says 'arr' frequently.")

bot.chat("Hello!")
bot.chat("What's your favorite food?")
bot.chat("Tell me about the sea.")

### Pattern 2: Save/Load Conversations

In [ ]:
import json

class PersistentConversation:
    """Conversation that can be saved and loaded."""
    def __init__(self, filepath: str = None):
        self.filepath = filepath
        self.conversation = ConversationManager()
        
        if filepath and os.path.exists(filepath):
            self.load()
    
    def save(self):
        """Save conversation to file."""
        with open(self.filepath, 'w') as f:
            json.dump(self.conversation.messages, f, indent=2)
        print(f"💾 Saved to {self.filepath}")
    
    def load(self):
        """Load conversation from file."""
        with open(self.filepath, 'r') as f:
            self.conversation.messages = json.load(f)
        print(f"📂 Loaded from {self.filepath}")
    
    def chat(self, message: str) -> str:
        response = self.conversation.send(message)
        if self.filepath:
            self.save()  # Auto-save after each message
        return response

# Example usage
conv = PersistentConversation("my_conversation.json")
conv.chat("Remember, my favorite color is blue.")
conv.chat("What's my favorite color?")

# Later, you can reload and continue
# new_session = PersistentConversation("my_conversation.json")
# new_session.chat("Do you remember my favorite color?")

## Best Practices

### 1. Always include system prompt
```python
messages = [
    {"role": "system", "content": "Your behavior instructions"},
    # ... conversation
]
```

### 2. Track token usage
```python
total_tokens = sum(len(encoding.encode(msg['content'])) for msg in messages)
```

### 3. Implement context window limits
Don't let conversations grow unbounded!

### 4. Handle errors gracefully
```python
try:
    response = client.chat.completions.create(...)
except Exception as e:
    # Handle error, maybe retry
```

### 5. Consider conversation summarization
For very long conversations, periodically summarize and start fresh.

## Practice Exercises

1. Build a chatbot that remembers user preferences across multiple turns
2. Implement a conversation manager that keeps only the last 5 message pairs
3. Create a bot that tracks how many turns have passed and mentions it
4. Build a Q&A bot that references previous questions in its answers
5. Implement conversation branching (save state, try different paths)